# Representations of Color
Converting input Hex codes to RGB and HSL.

In [1]:
# Assuming input six values
entries = input("Enter list of colors (#FF0000, #00ff00, #fff):").split()
entries

['ff0000']

In [2]:
def hexFormat(s:str):

    s = s.replace('#','')
    
    if len(s) == 3:
        s = [s[i] * 2 for i in range(3)] 

    return ''.join(s)

In [3]:
colors = [hexFormat(s) for s in entries]
colors

['ff0000']

## Hex to RGB
Convert every set of two hexidecimals to decimals.

In [4]:
def hexToRGB(h:str):
    # hexadecimal to decimal
    return [int(h[i:i+2], 16) for i in (0, 2, 4)]

In [5]:
rgb_colors = [hexToRGB(i) for i in colors]
rgb_colors

[[255, 0, 0]]

## RGB to HSL
This human-friendly conversion allows us to represent color on a color wheel by determining where the color point exists in relation to pure red, green, or blue. We can do this in 4 steps:

**Step 1**. Calculate the max, min, and chroma.  

$
M = max(R, G, B) \\
m = min(R, G, B) \\
C = range(R, G, B) = M - m$

**Step 2**. Calculate the hue measured in degrees `[0°, 360°)`.  

$H' = 
\begin{cases}
\text{undefined}, & \text{if } C = 0 \\
\frac{G-B}{C} \bmod 6, & \text{if } M = R \\
\frac{B-R}{C} + 2, & \text{if } M = G \\
\frac{R-G}{C} + 4, & \text{if } M = B
\end{cases}$

Then, $H = 60° × H'$

**Step 3**. Calculate Lightness measuring color intensity using set `[0%, 100%)`.

$L = mid(R, G, B) = \frac{1}{2}(M + m)$
    

**Step 4**. Calculate Saturation determined by the amount of grayscale or pure color from `[0%, 100%)`.

$S_L =
\begin{cases}
0, & \text{if } L = 1 \text{ or } L = 0 \\
\frac{C}{1-|2L-1|}, & \text{otherwise}
\end{cases}$
  

See [wiki page](https://en.wikipedia.org/wiki/HSL_and_HSV).

In [6]:
def rgbToHSL(rgb:list):
    # Change range from 0-255 to 0-100%
    r = rgb[0] / 255
    g = rgb[1] / 255
    b = rgb[2] / 255

    # Determine Chroma component values 
    cmax = max(r, g, b)
    cmin = min(r, g, b)
    delta = cmax - cmin

    # Calculate Hue: Distance from white (0-350 degrees)
    if delta == 0:
        h = 0
    elif cmax == r:                # more red
        h = (g - b) / delta % 6
    elif cmax == g:
        h = (b - r) / delta + 2.0   # more green
    else:                       # more blue
        h = (r - g) / delta + 4.0

    # Calulate Lightness: Midrange of RGB (0-100%)
    l = (cmax + cmin) / 2

    # Calulate Saturation: Gray to Pure (0-100%)
    if delta == 0:
        s = 0
    else:
        s = delta / (1 - abs(2 * l - 1))
    

    h = round(h * 60)
    s = round(s, 2)
    l = round(l, 2)
    return [h, s, l]

In [7]:
hsl_colors = [rgbToHSL(i) for i in rgb_colors]
hsl_colors

[[0, 1.0, 0.5]]

# Lightness Adjustments
Center the base lightness value and generate a range of shades and tints in the same hue. Tints add light to the lightness (`l`). Shades removes light from the lightness (`l`). Functions assume that l is a decimal value. The base color value is included in the final output.

In [99]:
# Fixed lightness values from 5% to 90%
def sequence(hsl_values:list, n=10, min_l=10, max_l=90) -> list:
    h = round(hsl_values[0]*100)
    s = round(hsl_values[1]*100)
    l = round(hsl_values[2]*100)
    
    shade_diff = int((l - min_l) / (n/2))
    shades = [i for i in range(min_l, l, shade_diff)]

    tint_diff = int((max_l - l) / (n/2))
    tints = [i for i in range(l, max_l, tint_diff)]

    return [[h,s,l/100] for l in shades+tints]

In [106]:
[sequence(hsl_colors[i]) for i in range(len(hsl_colors))]

[[[0, 100, 0.1],
  [0, 100, 0.18],
  [0, 100, 0.26],
  [0, 100, 0.34],
  [0, 100, 0.42],
  [0, 100, 0.5],
  [0, 100, 0.58],
  [0, 100, 0.66],
  [0, 100, 0.74],
  [0, 100, 0.82]]]

# Complimentary Colors
Colors compliment each other when they are directly opposite one another on the color wheel. Add $180\degree$ to the base color hue value.

In [101]:
def compliment(hsl_values:list):
    h = hsl_values[0]
    s = hsl_values[1]
    l = hsl_values[2]

    if (h + 180) > 360:
        return [h-180, s, l]
    else:
        return [h+180, s, l]

In [102]:
[compliment(hsl_colors[i]) for i in range(len(hsl_colors))]

[[180, 1.0, 0.5]]

# HSL to Hex
Returning to machine compatible colors.

### HSL to RGB
Step 1. Find chroma.

$C = (1 - |2L - 1|) * S_L$
  

Step 2. Find a point (R,G,B) with the same hue and chroma.

$H' = \frac{H}{60\degree}$
    
$X = C s (1 |H' \mod 2 - 1|) \\$  
 
$(R_1, G_1, B_1) = 
\begin{cases}
(C, X, 0) & \text{if } 0<=H'<1 \\
(X, C, 0) & \text{if } 1<=H'<2 \\
(0, C, X) & \text{if } 2<=H'<3 \\
(0, X, C) & \text{if } 3<=H'<4 \\
(X, 0, C) & \text{if } 4<=H'<5 \\
(C, 0, X) & \text{if } 5<=H'<6 \\
\end{cases}$

Step 3. Add amount to match the lightness.

$m = L - \frac{C}{2}$  
 
$(R, G, B) = (R_1 + m, G_1 + m, B_1 + m)$

In [ ]:
def hslToRGB(colors:list):
    h = colors[0]
    s = colors[1]
    l = colors[2]

    # Find chroma
    chroma = (1 - abs(2*l - 1)) * s

    # Find point (r,g,b) of RGB cube
    X = chroma * (1 - abs((h/60) % 2 - 1))

    if      0 <= h < 1: rgb = [chroma, X, 0]
    elif    1 <= h < 2: rgb = [X, chroma, 0]
    elif    2 <= h < 3: rgb = [0, chroma, X]
    elif    3 <= h < 4: rgb = [0, X, chroma]
    elif    4 <= h < 5: rgb = [X, 0, chroma]
    else:   rgb = [chroma, 0, X]

    # Match the lightness
    m = l - chroma/2
    return [int((i+m) * 255) for i in rgb]

In [ ]:
rgb_colors2 = [hslToRGB(i) for i in hsl_colors]
rgb_colors2

[[255, 0, 0]]

### RGB to Hex

In [ ]:
def rgbToHex(colors:list):
    hexcode = [hex(i)[2:] for i in colors]

    # six digit format
    for i in range(len(hexcode)):
        if len(hexcode[i]) != 2:
            hexcode[i] *= 2 
    
    return ''.join(hexcode)

In [ ]:
hex_colors = [rgbToHex(i) for i in rgb_colors2]
hex_colors

['ff0000']